In [39]:
import os
import warnings
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
import gradio as gr

In [40]:
# Suppress pygame welcome message
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
warnings.filterwarnings("ignore")

In [41]:
endpoint = ""
model_name = ""
token = ""

In [42]:
# Function to generate response using Azure's Phi-4 model
def generate_response(msg):
    client = ChatCompletionsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token),
    )
    response = client.complete(
        messages=[UserMessage(msg)],
        temperature=1.0,   # Controls the randomness of the output
        top_p=1.0,         # Controls the diversity of the response
        max_tokens=1000,   # Limit the length of the response
        model=model_name   # Model you're using, Phi-4 in this case
    )
    return response.choices[0].message.content

In [43]:
def explain_code(code):
    """Generate an explanation for the provided code."""
    if not code.strip():
        return "Please enter some code to explain."
    
    prompt = f"""Explain this Python code in detail:
    {code}

Explanation:
1. Overall purpose:
2. Key components:
3. Flow of execution:
4. Notable features:"""
    
    return generate_response(prompt)

In [50]:
def create_interface():
    with gr.Blocks(title="Code Explainer", theme="soft") as interface:
        gr.Markdown("# AI Tutor For Programming")
        with gr.Row():
            with gr.Column():
                code_input = gr.Textbox(
                    label="Code/Prompt",
                    placeholder="Paste your code here...",
                    lines=10,
                    elem_id="code-input"
                )
                with gr.Row():
                    submit_btn = gr.Button("Submit", variant="primary")
                    clear_btn = gr.Button("Clear")
            
            with gr.Column():
                explanation_output = gr.Textbox(
                    label="Explanation",
                    lines=15,
                    interactive=False,
                    elem_id="explanation-output"
                )
        submit_btn.click(
            fn=explain_code,
            inputs=code_input,
            outputs=explanation_output
        )
        clear_btn.click(
            fn=lambda: ("", ""),
            inputs=None,
            outputs=[code_input, explanation_output]
        )
    return interface

In [51]:
if __name__ == "__main__":
    # Create and launch the interface
    interface = create_interface()
    interface.launch()

* Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.
